In [4]:
# from pyml4.common import context, db
# context.local_context.print_summary()
from __future__ import print_function
from os.path import dirname
import os,sys,inspect

import gensim
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import pymysql.cursors
import json

current_dir = dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
manage_dir = dirname(dirname(dirname(current_dir)))
if not manage_dir in sys.path:
    sys.path.insert(0, manage_dir)

In [7]:
def find_capitalization_dup(data, tag_col):
    new_data = data.copy()
    tag_lowercase_list = []
    tag_lowercase_dup_dict = {}

    # add a column that contains the lowercase version of tags
    for index, row in new_data.iterrows():
        tag = row[tag_col]
        tag_lower = tag.lower()
        tag_lowercase_list.append(tag_lower)
        if tag_lower not in tag_lowercase_dup_dict:
            tag_lowercase_dup_dict[tag_lower] = []
            tag_lowercase_dup_dict[tag_lower].append(tag)
        else:
            tag_lowercase_dup_dict[tag_lower].append(tag)

    new_data['tag_lowercase'] = tag_lowercase_list

    tag_lowercase_dup_list = []
    # identify capitalization duplicates
    print("identify capitalization duplicates")
    for index, row in new_data.iterrows():
        tag = row[tag_col]
        tag_lowercase = row[-1]
        temp = ""
        for item in tag_lowercase_dup_dict[tag_lowercase]:
            # print(tag_lowercase)
            #print(item)
            if item != tag:
                temp += item + ";"
        if temp != "":
            tag_lowercase_dup_list.append(temp)
        else:
            tag_lowercase_dup_list.append(None)
    # drop duplicates with lower scores
    new_data["tag_lowercase_dups"] = tag_lowercase_dup_list
    return new_data

In [31]:
def find_punctuation_dup(data, tag_col):
    new_data = data.copy()
    print("start processing tags with punctuations")
    import string

    table = str.maketrans('', '', string.punctuation)

    tag_strip_punc = []
    tag_punctuation_dup_dict = {}

    # Strip punctuation and store tag - tag_no_punctuation pairs
    for index, row in new_data.iterrows():
        tag = row[0]
        tag_lowercase = tag.lower()
        
        stripped_tag = tag_lowercase.translate(table)
        tag_strip_punc.append(stripped_tag)
        if stripped_tag not in tag_punctuation_dup_dict:
            tag_punctuation_dup_dict[stripped_tag] = []
            tag_punctuation_dup_dict[stripped_tag].append(tag)
        else:
            tag_punctuation_dup_dict[stripped_tag].append(tag)
    new_data['tag_strip_punctuation'] = tag_strip_punc

    tag_punct_dup_list = []

    # print(data.head())
    
    for index, row in new_data.iterrows():
        tag = row[0]

        # make this a parameter
        tag_no_punc = row[-1]
        temp = ""
        for item in tag_punctuation_dup_dict[tag_no_punc]:
            # print(tag_lowercase)
            #print(item)
            if item != tag:
                temp += item + ";"
        if temp != "":
            tag_punct_dup_list.append(temp)
        else:
            tag_punct_dup_list.append(None)

    new_data["tag_punc_dups"] = tag_punct_dup_list
    return new_data

In [39]:
## TODO

def remove_whitespace_dup(data, tag_col):
    import string
    new_data = data.copy()
    table = str.maketrans('', '', string.punctuation)
    print("start removing spaces")
    
    tag_no_spaces_list = []
    tag_no_spaces_dup_dict = {}

    # Strip punctuation and store tag - tag_no_punctuation pairs
    for index, row in new_data.iterrows():
        tag = row[0]
        tag_lower = tag.lower()
        tag_no_cap_punc = tag_lower.translate(table)
        tag_no_space = tag_no_cap_punc.replace(" ", "")

        tag_no_spaces_list.append(tag_no_space)
        if tag_no_space not in tag_no_spaces_dup_dict:
            tag_no_spaces_dup_dict[tag_no_space] = []
            tag_no_spaces_dup_dict[tag_no_space].append(tag)
        else:
            tag_no_spaces_dup_dict[tag_no_space].append(tag)
    new_data['tag_no_whitespace'] = tag_no_spaces_list

    tag_no_spaces_dup_list = []
    
    for index, row in new_data.iterrows():
        tag = row[0]
        
        # make this a parameter
        tag_no_whitespace = row[-1]
        temp = ""
        for item in tag_no_spaces_dup_dict[tag_no_whitespace]:
            # print(tag_lowercase)
            #print(item)
            if item != tag:
                temp += item + ";"
        if temp != "":
            tag_no_spaces_dup_list.append(temp)
        else:
            tag_no_spaces_dup_list.append(None)

    new_data["tag_no_spaces_dups"] = tag_no_spaces_dup_list
    return new_data

In [10]:
def isEditDistanceTwo(s1, s2):

    # Find lengths of given strings
    m = len(s1)
    n = len(s2)

    # If difference between lengths is more than 1,
    # then strings can't be at one distance
    if abs(m - n) > 1:
        return False
    count = 0    # Count of isEditDistanceOne
    i = 0
    j = 0
    while i < m and j < n:
        # If current characters dont match
        if s1[i] != s2[j]:
            if count == 1:
                return False
            # If length of one string is
            # more, then only possible edit
            # is to remove a character
            if m > n:
                i+=1
            elif m < n:
                j+=1
            else:    # If lengths of both strings is same
                i+=1
                j+=1
            # Increment count of edits
            count+=1
        else:    # if current characters match
            i+=1
            j+=1
    # if last character is extra in any string
    if i < m or j < n:
        count+=1
    return count <= 2

In [11]:
def word2vec_and_2edits_away(data):
    print("start finding synonyms using word2vec")
    new_data = data.copy()
    tag_list = new_data['tag'].tolist()

    # load word2vedc model; edit the path and filename below;
    word_vec_model2 = Word2Vec.load('../models/Word2Vec_fullds_model2')

    # find synonyms using word2vec model;
    word2vec = []
    word2vec_2edits = []

    for index, row in new_data.iterrows():
        tag = row[0]
        temp1 = ''
        temp2 = ''
        if tag in word_vec_model2.wv.vocab:
            for (k,v) in word_vec_model2.wv.similar_by_word(tag):
                if k in tag_list:
                    #temp.append(k)
                    temp1 += k+";"
                    if isEditDistanceTwo(tag,k):
                        temp2 += k+";"
        if len(temp1) > 0:
            word2vec.append(temp1)
        else:
            word2vec.append(None)
        if len(temp2) > 0:
            word2vec_2edits.append(temp2)
        else:
            word2vec_2edits.append(None)
    # data = tag_no_cap_punc_stem.copy()
    new_data['word2vec'] = word2vec
    new_data['word2vec_2edits'] = word2vec_2edits
    return new_data

In [54]:
# Insert into mysql server:
def insert_to_sql(data_in_tuple, sql_table_name):
    ### Connect and insert to SQL

    import pymysql.cursors
    connection = pymysql.connect(host="127.0.0.1",
                                 user="web",
                                 password="atth1132",
                                 db="ML3_dev")
    try:
        with connection.cursor() as cursor:
            sql_drop = "DELETE from %s" % (sql_table_name)
            cursor.execute(sql_drop)
            sql = "insert ignore into tag_replacement(tag,word2vec_2edits,tag_lowercase_dups,tag_punctuation_dups,tag_no_spaces_dups) VALUES(%s, %s, %s, %s, %s)"
            #number_of_rows = cursor.executemany(sql, data_to_insert)
            #% (tag_list,score_list,duplicates_list,word2vec_2edits_list,duplicates_with_word2vec_list)
            #for x in tuples:
            cursor.executemany(sql,data_in_tuple)
            #   cursor.execute(sql,x)

            connection.commit()
    finally:
        connection.close()

In [13]:
# load tag_movie from sql database
connection = pymysql.connect(host="127.0.0.1",
                             user="web",
                             password="atth1132",
                             db="ML3_mirror")

try:
    with connection.cursor() as cursor:
        sql = "SELECT tag,score FROM tag_movie"
        cursor.execute(sql)
        result = cursor.fetchall()

finally:
    connection.close()

movie_tag_app = pd.DataFrame(list(result), columns=['tag','score'])
movie_tag_app_agg = movie_tag_app.groupby("tag").sum().reset_index()


###### find capitalization dup
tag_capitalization_dups = find_capitalization_dup(movie_tag_app_agg, 0)
#tag_capitalization_dups.sort_values(by="score",ascending=False,inplace=True)
#tag_capitalization_dups.sort_values(by="tag_lowercase",inplace=True)

print("drop duplicates with lower scores")
#tag_no_capitalization_dup = tag_capitalization_dups.drop_duplicates("tag_lowercase",keep="first")




identify capitalization duplicates
drop duplicates with lower scores


In [25]:
tag_capitalization_dups.sort_values(by=["tag_lowercase","score"],inplace=True)
tag_capitalization_dups.drop_duplicates("tag_lowercase", keep="last", inplace=True)

,tag,score,tag_lowercase,tag_lowercase_dups
1521,Alcoholism,3.0,alcoholism,ALCOHOLISM;alcoholism;
28466,alcoholism,247.0,alcoholism,ALCOHOLISM;Alcoholism;
1522,Alden Ehrenreich,1.0,alden ehrenreich,None
1523,Alden Indy Boots,1.0,alden indy boots,None
1524,"Aldo, Giovanni e Giacomo",4.0,"aldo, giovanni e giacomo",None
1525,Aldous Huxley,0.0,aldous huxley,None
28467,alec baldwin,-1.0,alec baldwin,Alec Baldwin;
1526,Alec Baldwin,7.0,alec baldwin,alec baldwin;
1527,Alec Guiness,-11.0,alec guiness,alec guiness;
28468,alec guiness,-1.0,alec guiness,Alec Guiness;


In [32]:
tag_no_cap_punc_dup = find_punctuation_dup(tag_capitalization_dups, 0)

start processing tags with punctuations


,tag,score,tag_lowercase,tag_lowercase_dups,tag_strip_punctuation,tag_punc_dups
0,!950's Superman TV show,-4.0,!950's superman tv show,None,950s superman tv show,None
1,!David O. Russell,-8.0,!david o. russell,None,david o russell,David O. Russell;
2,!George Clooney,-9.0,!george clooney,None,george clooney,George Clooney;
3,!George Lucas,-9.0,!george lucas,None,george lucas,George Lucas;George Lucas.;
4,"""A Christmas Carol""",0.0,"""a christmas carol""",None,a christmas carol,A Christmas Carol;
5,"""A MÃ£o-de-Deus""",-1.0,"""a mã£o-de-deus""",None,a mã£odedeus,None
6,"""American"" French",-1.0,"""american"" french",None,american french,None
23,"""artsy""",2.0,"""artsy""",None,artsy,artsy;
25,"""bad cia""-- too simplistic",0.0,"""bad cia""-- too simplistic","""bad CIA""-- too simplistic;",bad cia too simplistic,None
26,"""based on true events""",2.0,"""based on true events""",None,based on true events,based on true events;


In [33]:
tag_no_cap_punc_dup.sort_values(by=["tag_strip_punctuation","score"],inplace=True)
tag_no_cap_punc_dup.drop_duplicates("tag_strip_punctuation", keep="last", inplace=True)
tag_no_cap_punc_dup.sort_values(by="score",ascending=False)

,tag,score,tag_lowercase,tag_lowercase_dups,tag_strip_punctuation,tag_punc_dups
62280,sci-fi,7866.0,sci-fi,Sci-Fi;Sci-fi;,scifi,#scifi;+scifi;scifi;
29971,atmospheric,5424.0,atmospheric,ATMOSPHERIC;Atmospheric;,atmospheric,None
66894,surreal,4598.0,surreal,Surreal;,surreal,None
27949,action,4538.0,action,ACTION;Action;,action,None
69686,twist ending,4120.0,twist ending,Twist Ending;Twist ending;,twist ending,None
71124,visually appealing,3875.0,visually appealing,Visually Appealing;Visually appealing;,visually appealing,None
39734,dystopia,3841.0,dystopia,Dystopia;,dystopia,None
35887,comedy,3821.0,comedy,COMEDY;Comedy;,comedy,#comedy;Comedy??;
72391,woman director,3651.0,woman director,Woman director;,woman director,None
37659,dark comedy,3513.0,dark comedy,Dark Comedy;Dark comedy;,dark comedy,None


In [40]:
tag_no_cap_punc_space_dup = remove_whitespace_dup(tag_no_cap_punc_dup, 0)
tag_no_cap_punc_space_dup.sort_values(by="score",ascending=False)

start removing spaces


,tag,score,tag_lowercase,tag_lowercase_dups,tag_strip_punctuation,tag_punc_dups,tag_no_whitespace,tag_no_spaces_dups
62280,sci-fi,7866.0,sci-fi,Sci-Fi;Sci-fi;,scifi,#scifi;+scifi;scifi;,scifi,sci fi;
29971,atmospheric,5424.0,atmospheric,ATMOSPHERIC;Atmospheric;,atmospheric,None,atmospheric,None
66894,surreal,4598.0,surreal,Surreal;,surreal,None,surreal,None
27949,action,4538.0,action,ACTION;Action;,action,None,action,None
69686,twist ending,4120.0,twist ending,Twist Ending;Twist ending;,twist ending,None,twistending,twist-ending;
71124,visually appealing,3875.0,visually appealing,Visually Appealing;Visually appealing;,visually appealing,None,visuallyappealing,None
39734,dystopia,3841.0,dystopia,Dystopia;,dystopia,None,dystopia,None
35887,comedy,3821.0,comedy,COMEDY;Comedy;,comedy,#comedy;Comedy??;,comedy,None
72391,woman director,3651.0,woman director,Woman director;,woman director,None,womandirector,None
37659,dark comedy,3513.0,dark comedy,Dark Comedy;Dark comedy;,dark comedy,None,darkcomedy,darkcomedy;


In [41]:
tag_no_cap_punc_space_dup.sort_values(by=["tag_no_whitespace","score"],inplace=True)
tag_no_cap_punc_space_dup.drop_duplicates("tag_no_whitespace", keep="last", inplace=True)
tag_no_cap_punc_space_dup.sort_values(by="score",ascending=False)
tag_no_cap_punc_space_dup

,tag,score,tag_lowercase,tag_lowercase_dups,tag_strip_punctuation,tag_punc_dups,tag_no_whitespace,tag_no_spaces_dups
791,=======================,2.0,=======================,None,,************************;+++++++++++++++;.;:);...,,None
191,007,182.0,007,None,007,None,007,None
195,007-like,0.0,007-like,None,007like,None,007like,None
193,007 movies are bad.,0.0,007 movies are bad.,None,007 movies are bad,None,007moviesarebad,None
194,007 rocks,-5.0,007 rocks,None,007 rocks,None,007rocks,None
192,007 (series),16.0,007 (series),None,007 series,None,007series,None
196,01.02.05,-4.0,01.02.05,None,010205,None,010205,None
197,01.03.06,-6.0,01.03.06,None,010306,None,010306,None
198,01.04.06,-7.0,01.04.06,None,010406,None,010406,None
200,01/10,-275.0,01/10,None,0110,None,0110,None


In [57]:
def main():
    # load tag_movie from sql database
    connection = pymysql.connect(host="127.0.0.1",
                                 user="web",
                                 password="atth1132",
                                 db="ML3_dev")

    try:
        with connection.cursor() as cursor:
            sql = "SELECT tag,score FROM tag_movie"
            cursor.execute(sql)
            result = cursor.fetchall()

    finally:
        connection.close()

    movie_tag_app = pd.DataFrame(list(result), columns=['tag','score'])
    movie_tag_app_agg = movie_tag_app.groupby("tag").sum().reset_index()
    tag_no_cap_dup = find_capitalization_dup(movie_tag_app_agg,0)
    # Note that find_punctuation_dup uses tag_lowercase;
    # remove_whitespace_dup uses tag_no_cap_punc_dups;
    tag_no_cap_dup.sort_values(by=["tag_lowercase","score"],inplace=True)
    print("drop duplicates when tags are converted to lowercase")
    tag_no_cap_dup.drop_duplicates("tag_lowercase",keep="last",inplace=True)
    
    tag_no_cap_punc_dup = find_punctuation_dup(tag_no_cap_dup,0)
    #tag_no_cap_punc_space_dup2.sort_values(by="score",ascending=False,inplace=True)
    tag_no_cap_punc_dup.sort_values(by=["tag_strip_punctuation","score"],inplace=True)
    # print("drop duplicates when punctuations are removed")
    tag_no_cap_punc_dup.drop_duplicates("tag_strip_punctuation",keep="last",inplace=True)
    
    
    tag_no_cap_punc_space_dup = remove_whitespace_dup(tag_no_cap_punc_dup,0)
    tag_no_cap_punc_space_dup.sort_values(by=["tag_strip_punctuation","score"],inplace=True)
    tag_no_cap_punc_space_dup.drop_duplicates("tag_strip_punctuation", keep="last", inplace=True)

    
    tag_no_cap_pun_space_dup_word2vec = word2vec_and_2edits_away(tag_no_cap_punc_space_dup)
    
    print(tag_no_cap_punc_space_dup.head())
    #### Insert into SQL server
    data_to_insert = tag_no_cap_pun_space_dup_word2vec[["tag","word2vec_2edits","tag_lowercase_dups","tag_punc_dups","tag_no_spaces_dups"]].copy()
    
    # convert np.nan to None type in python
    data_to_insert = data_to_insert.where(data_to_insert.notnull(), None)
    tuples = [tuple(x) for x in data_to_insert.values]
    insert_to_sql(tuples, "tag_replacement")
    

In [58]:
main()

identify capitalization duplicates
drop duplicates when tags are converted to lowercase
start processing tags with punctuations
start removing spaces
start finding synonyms using word2vec


/Users/evazhong/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


                         tag  score            tag_lowercase  \
756  =======================    2.0  =======================   
742                ; bigoted   -1.0                ; bigoted   
185                  0 stars   -4.0                  0 stars   
186                      007  163.0                      007   
188      007 movies are bad.    0.0      007 movies are bad.   

    tag_lowercase_dups tag_strip_punctuation  \
756               None                         
742               None               bigoted   
185               None               0 stars   
186               None                   007   
188               None    007 movies are bad   

                                         tag_punc_dups tag_no_whitespace  \
756  ************************;+++++++++++++++;.;:);...                     
742                                               None           bigoted   
185                                               None            0stars   
186                   

/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1265, "Data truncated for column 'tag_punctuation_dups' at row 1")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE1\\xBB\\x99 ph...' for column 'tag' at row 7170")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xC4\\xB1lmaz' for column 'tag' at row 7791")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xC8\\x99in\\xC4\\x83...' for column 'tag' at row 8491")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xC2\\x83?\\xC3\\x83\\xC2...' for column 'tag' at row 8500")
  result = self._query(q